In [ ]:
import numpy as np                            # numpy is for math (it's hyper fast)
from plot_robot import *                      # custom plotting functions for robot
from kinematics_full import *                 # custom kinematic functions
from cspline import *                         # custom cubic spline functions

In [ ]:
def get_dh_param_table(q):
    '''
    Returns a table containing the Denavit-Hartenberg parameters,
    with the row i having the following pattern: [theta_i, alpha_i, r_i, d_i].
    In contrast to the lecture, i starts at zero (because informatics..)
    The values of this table depend on the robot's current configuration q.

    Also the indices of the configuraion vector q start at zero.
    Thus, what we called theta_1 in the lecture is here q[0].
    '''
    LINK_LENGTH_1  = 0.4
    LINK_LENGTH_2  = 0.3
    LINK_LENGTH_3  = 0.3   
    LINK_LENGTH_4  = 0.05
    
    return np.array([
        [q[0],      0.0,  LINK_LENGTH_1,     0.0],
        [q[1],      0.0,  LINK_LENGTH_2,     0.0],
        [q[2],      0.0,  LINK_LENGTH_3,     0.0],
        [q[3],      0.0,  LINK_LENGTH_4,     0.0],
    ])
# ---

In [ ]:
def get_env_border(x: float) -> float:
    '''
    Defines the border of the environment which the
    end effector of the robot cannot pass through.

    :param x:     horizontal position on border
    :return:      vertical position on border that corresponds to x
    '''

    return 0.1 * np.sin(x*10) - 0.5
# ---

### Virtual Trajectory

Task: Create a virtual c-spline trajectory that passes through the environment.

In [ ]:
# specify initial robot configuration
q_init = np.array([np.pi - np.pi/4, np.pi/4, np.pi/4, 0])

# specify keyframes
target_poses = np.array([
                            forward_kinematics(get_dh_param_table(q_init)),
                            '''<IMPLEMENT YOUR CODE HERE!>'''
                        ])
 
# specify duration of keyframe interpolations
durations = np.array([5, 2]) 


# generate cubic spline
coeffs        = create_multi_point_cspline_interpolation(target_poses, durations)
n_pts         = 100
ts            = np.linspace(0, np.sum(durations), n_pts)
pos, vel, acc = eval_multi_point_spline_interpolation(ts, target_poses, durations, coeffs)

plot_robot_for_frames(frames=get_frames(get_dh_param_table(q_init)), target_poses=target_poses, interp_xs=pos[:, 0], interp_ys=pos[:, 1], plot_env=True, get_env_border=get_env_border, ylim=(-1., 0.4), xlim=(-1.2, 0.6))

In [ ]:
# reset robot
q          = q_init.copy()
dh_params  = get_dh_param_table(q)

old_ee_pos = curr_ee_pos = forward_kinematics(dh_params)
old_ee_vel = curr_ee_vel = np.zeros(6)
old_ee_acc = curr_ee_acc = np.zeros(6)
f_ext      = np.zeros(6)

fs_ext     = []
fs_m       = []
fs_b       = []
fs_k       = []

old_target_vel = 0


M = 100
B = 1
K = 10

t  = 1
dt = 1

### Impedance

With impedance control, the end effector realizes an impedance to the environment that can be modelled as a spring-mass-damper system:

$$ -F_{ext} = M(\ddot{x} - \ddot{x}_d) + B(\dot{x} - \dot{x}_d) + K(x - x_d) $$
$$ F_{ext} = -F_\mathrm{inertia} - F_\mathrm{friction} - F_\mathrm{stiffness} $$

where $F_{ext}$ denotes the contact force between end effector and environment, $M$ is the inertia matrix, the matrix $B$ encodes damping, and $K$ stiffness. Here, $x$ refers to the actual end effector pose and $x_d$ denotes the desired pose. 

In [ ]:
'''
You need to run this code cell multiple times to iteratively move the robot's end effector.
For this, press <CTRL> + <ENTER> multiple times on Linux or Windows and <CMD> + <ENTER> on Mac.
'''


# determine target values
target_pos     = pos[np.min([t, len(pos)-1])].flatten()
target_vel     = pos[np.min([t, len(pos)-1])].flatten() - pos[np.min([t-1, len(pos)-1])].flatten()
target_acc     = target_vel - old_target_vel
old_target_vel = target_vel

# remember previous robot state
old_ee_pos = curr_ee_pos
old_ee_vel = curr_ee_vel
old_ee_acc = curr_ee_acc

# move robot so that it does not pass the environment
mov_target_pose = target_pos.copy()
mov_target_pose[1] = np.max([get_env_border(mov_target_pose[0]), mov_target_pose[1]])
q += get_step(mov_target_pose, dh_params)
dh_params   = get_dh_param_table(q)

# get current robot state
curr_ee_pos = forward_kinematics(dh_params)
curr_ee_vel = curr_ee_pos - old_ee_pos
curr_ee_acc = curr_ee_vel - old_ee_vel

# compute impedance
f_inertia   = '''<IMPLEMENT YOUR CODE HERE!>'''
f_friction  = '''<IMPLEMENT YOUR CODE HERE!>'''
f_stiffness = '''<IMPLEMENT YOUR CODE HERE!>'''
f_ext       = '''<IMPLEMENT YOUR CODE HERE!>'''

# keep track of exerted vertical force
fs_ext.append(f_ext[1])
fs_m.append(f_inertia[1])
fs_b.append(f_friction[1])
fs_k.append(f_stiffness[1])

t += dt


plot_robot_for_frames(frames=get_frames(dh_params), target_poses=target_poses, interp_xs=pos[:, 0], interp_ys=pos[:, 1], plot_env=True, get_env_border=get_env_border, ylim=(-1., 0.4), xlim=(-1.2, 0.6))

### Exerted Forces
The contact forces $F_{ext}$ between the robot's end effector and the environment can be decomposed into forces due to inertia, friction and stiffness. Investigate what is going on.

In [ ]:
plt.figure(figsize=(5,2), dpi=150)
plt.title('Exerted contact force')
plt.plot(durations.sum() * np.arange(len(fs_ext)) / n_pts, np.abs(np.array(fs_ext)))
plt.xlabel('Time [s]')
plt.ylabel('$F_{ext}$')

plt.figure(figsize=(5,2), dpi=150)
plt.title('Inertial forces')
plt.plot(durations.sum() * np.arange(len(fs_m)) / n_pts, np.abs(np.array(fs_m)))
plt.xlabel('Time [s]')
plt.ylabel('$F_{inertia}$')

plt.figure(figsize=(5,2), dpi=150)
plt.title('Frictional forces')
plt.plot(durations.sum() * np.arange(len(fs_b)) / n_pts, np.abs(np.array(fs_b)))
plt.xlabel('Time [s]')
plt.ylabel('$F_{friction}$')

plt.figure(figsize=(5,2), dpi=150)
plt.title('Springy forces')
plt.plot(durations.sum() * np.arange(len(fs_k)) / n_pts, np.abs(np.array(fs_k)))
plt.xlabel('Time [s]')
plt.ylabel('$F_{stiffness}$')
plt.show()